# ML tutorial 1: Neural network classifiers for distinguishing signals from instrumental noise

Author: Melissa Lopez

Email: m.lopez@uu.nl

In this tutorial we will perform a supervised multi-classification task with the triggers from matched filtering. Our model differentiates an intermediate-mass binary black hole gravitational wave signal from different types of glitches.

If you are interested in learning more about the Machine Learning (ML) formalism, I recommend to read [Chapter 4 of my thesis](
  https://research-portal.uu.nl/ws/portalfiles/portal/245781352/phddissertationmelissalopez%20-%206739e41d4dc0a.pdf).

In [ ]:
import numpy as np
import sklearn
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau


### A short motivation to finding intermediate-mass binary black holes (IMBH)

The formation of supermassive black holes (SMBH) is not yet fully understood, despite their presence in nearly every galaxy, including the Milky Way.
As stellar evolution predicts, stars with a helium core mass in the range $\sim 32-64\,\text{M}_{\odot}$ leave no remnant due to pair-instability.
Because of this mass gap, a plausible explanation for the formation of (SMBH) is the hierarchical mergers of intermediate-mass black holes (IMBH). Therefore, direct observation of IMBH populations would strengthen the possible evolutionary link between stellar mass black holes and SMBH.

The detection of IMBHs in GW searches remained elusive until the detection of GW190521 during the third observing run.
Despite falling in the sensitivity band of current GWinterferometers, IMBH searches are challenging as few cycles of the signal can be observed with current ground-based detectors.
Moreover, they are also hampered by non-Gaussian transient noise of terretrial origin known as glitches.

### Extremely brief introduction to a GW search as a classification task

A search algorithm is a sequence of operations that inputs the raw data of the interferometer and produces a list of detection events or *triggers*, i.e. potential gravitational wave (GW) candidates, to solve the following hypothesis testing,

**Null hypothesis ($H_{0}$):** the time series $s(t)$ does not contain a \acrshort{gw} $\rightarrow P(h|s) < P_{*}$



**Alternative hypothesis ($H_{a_{0}}$):** the time series $s(t)$ contains a \acrshort{gw} $\rightarrow P(h|s) \geq P_{*}$



given a decision threshold $P_{*}$.

While state-of-the-art search pipelines construct the likelihood ratio $\Lambda$, we can build a supervised ML model that predicts the probability of the input data containing a GW signal based on past evidence, i.e. the model will learn the mapping $h: \mathcal{X} \rightarrow \mathcal{Y}$ between $\mathcal{X}$, the space of the detector's input $X$, and   $\mathcal{Y}$, the space of the targets $y$.

In ML such a supervised task can be defined as a classification problem, that can be *binary* or *multi-class*. If we perform a *binary*classification, we will distinguish between two classes: a *positive* class, usually represented by $y=1$, and a *negative*, usually represented by $y=0$. On the other hand, if we perform a *multi-class* classification, we will distinguish between $C$ classes.

Today we will performa a multi-class classification tasks where we can to differentiate simulated GW signals a.k.a. 'Injections' from non-Gaussian transient noise known as glitches. In particular, we have these classes:


`['Injections', 'Blip' , 'Koi_Fish', 'Low_Frequency_Burst', 'Tomte', 'Whistle','Fast_Scattering']`

To learn a mapping $h$ between the input data and these classes, the ML model minimizes the loss function known as *cross-entropy* via gradient descent. Mathematically,

\begin{equation}
\mathcal{L}(y, \hat{y}) = - \sum_{j}^{C} y_{j}\log{\hat{y}_{j}},
\end{equation}

where $\hat{y}$ represents the probability predicted by the model, $y$ is the target or ground truth and $C$ the number of clasess. Note that this is a generalization of *binary classification* loss function where $C=2$.


These loss functions have several desirable properties: they are easy to compute, differentiable and provide a probabilistic interpretation of the model's output. However, they are sensitive to class imbalance, which occurs when the defined classes have significantly different sizes of samples.


### Hands on tutorial

We can retrieve our dummy dataset with `wget`. This data set is a demo from the larger dataset used in [our paper](https://inspirehep.net/literature/2862532).





In [ ]:
!wget https://ldas-jobs.ligo.caltech.edu/~melissa.lopez/ML_course_mock/GlitchBank/mock_data/dataset_all_H1_bootstrap_O3a.npy

The dataset is a numpy matrix with shape [samples, variables]. Our variables in order are:

- **SNR**: Signal-to-Noise Ratio — measures how strong the signal is compared to the background noise. Higher values suggest stronger, more confident detections.
- **Chisq**: Chi-squared value — indicates how well the observed data matches the expected signal model. Lower values generally mean a better match.
- **Mass1**: Mass of the primary (heavier) compact object, typically given in solar masses.
- **Mass2**: Mass of the secondary (lighter) compact object, also in solar masses.
- **Spin1z**: Dimensionless spin component along the z-axis for the primary object — describes spin magnitude and orientation relative to the orbital angular momentum.
- **Spin2z**: Dimensionless spin component along the z-axis for the secondary object.
- **Class**: Label indicating the type of event — used in classification tasks to distinguish between signal and various types of noise or glitches. The mapping is as follows:
  - 'Injections' → `0` (simulated intermediate-mass black hole signal)
  - 'Blip' → `1`
  - 'Koi_Fish' → `2`
  - 'Low_Frequency_Burst' → `3`
  - 'Tomte' → `4`
  - 'Whistle' → `5`
  - 'Fast_Scattering' → `6`

**Exercise 1:** Load the dataset.
- How many injections are there? How many Blips? Why?
- What is the minimum Mass1? And the maximum?

The input $X$ to our model will be the `data` matrix with size `[samples, variables - 1]`, because the last variable in the matrix is the target $y$.
Since our task is multi-class and not binary classification we have consider:
- *Categorical target definition:* In a multi-class setting, the target variable must be encoded in a format suitable for distinguishing among multiple classes. This is typically done using one-hot encoding. This allows the model to output a probability distribution over all possible classes.

- *Softmax Activation in the Output Layer:* Instead of using a sigmoid activation function, which is suitable for binary classification, we use a softmax activation function in the final layer. The softmax function  effectively produces a probability distribution across the multiple classes.


In [ ]:
X, y = data[:, :-1], data[:, -1]
# We transform the target into categorical since it is a multi-class task
output_classes = len(np.unique(y))
y = to_categorical(y, num_classes=output_classes)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=0)

Now, we want to build a simple multi-layer perceptron (MLP) with Keras.

Usually we split the data use for supervised classsification into `training`, `validation` and `testing`. `sklearn.model_selection.train_test_split` splits into 75 \% training and 25 \% testing. We still need to split the training into training and validation, which happens after the model is defined, in `model.fit` function, where we use 80\% for training and 20\% for validation.



> **Question:** why do we need training, validation and testing stages? Isn't enough to have training and testing?



With `Sequential` function we define a model with two dense or fully connected layers with `tanh` activation functions. As we mentioned before, the last layer outputs a `softmax` activation to produce a probability distribution of all classes.

With `compile` we call the training and validations stages that depend on the optimizer, the loss definition and the metrics for performance evaluation. Note that we usually use the minimization of the loss and the maximization of the accuracy as part of the same evaluation metric.




> **Question:** why do we need to examine loss AND accuracy to understand the behaviour of our model?



With `fit` we add all the ingredients from `Sequential` and `compile` together, as well as the number of epochs and batch size.

Now, we just excecute! Right?


In [ ]:
input_dim = X_train.shape[1]

# Create a simple MLP model
model = Sequential([
    Dense(32, activation='tanh', input_shape=(input_dim,)),
    Dense(16, activation='tanh'),
    Dense(output_classes, activation='softmax')])

# Compile the model
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_split=0.2)

I wrote some functions to help us visualize what is going on in the network.

In [ ]:
def plot_history(history):
  epochs = range(1, len(history.history['loss']) + 1)

  fig, ax1 = plt.subplots(figsize=(8, 5))

  # Plot loss on left y-axis
  ax1.plot(epochs, history.history['loss'], c='cornflowerblue', linestyle='solid', label='Train Loss')
  ax1.plot(epochs, history.history['val_loss'], c='cornflowerblue', linestyle='dashed', label='Val Loss')
  ax1.set_xlabel('Epoch')
  ax1.set_ylabel('Loss')
  ax1.tick_params(axis='y')

  # Create second y-axis for accuracy
  ax2 = ax1.twinx()
  ax2.plot(epochs, history.history['accuracy'], c='mediumseagreen', linestyle='solid', label='Train Acc')
  ax2.plot(epochs, history.history['val_accuracy'], c='mediumseagreen', linestyle='dashed', label='Val Acc')
  ax2.set_ylabel('Accuracy')
  ax2.tick_params(axis='y')

  # Title and legends
  fig.suptitle('Training and Validation Loss & Accuracy')
  lines1, labels1 = ax1.get_legend_handles_labels()
  lines2, labels2 = ax2.get_legend_handles_labels()
  ax1.legend(lines1 + lines2, labels1 + labels2, loc='center right')

  plt.tight_layout()
  plt.show()

  max_acc_train = history.history['accuracy'][-1]
  max_acc_val = history.history['val_accuracy'][-1]
  return max_acc_train, max_acc_val

def get_test_accuracy(history):

  y_pred_prob = model.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)
  y_true = np.argmax(y_test, axis=1)

  test_acc = accuracy_score(y_true, y_pred)
  cm = confusion_matrix(y_true, y_pred)
  return test_acc, cm

test_acc, cm = get_test_accuracy(history)
max_acc_train, max_acc_val = plot_history(history)
print(f'Train accuracy: {max_acc_train:.4f}, Validation accuracy: {max_acc_val:.4f}, Test accuracy: {test_acc:.4f}')

**Exercise 2:** What are the main problems with this network? Let's brainstorm a bit.

**Exercise 3:** Can we do better? Can we increase accuracy ~10%?

Some ideas:

- Scaling of the data. Why is it important?
- Another architecture?
- Learning rate scheduler?
- Batch size? Number of epochs?
- Regularization layers? e.g. Dropout

You can also use [Optuna](https://optuna.org/), but honestly I never use it.

Given this decision rule, there are two correct and two erroneous outcomes. If we correctly predict that a gravitational wave (GW) signal is present or absent in the time series \( s \), we refer to these cases as a *true positive* (TP) or *true negative* (TN), respectively. Conversely, if we incorrectly predict that a GW signal is present when it is not, this is called a *false positive* (FP), or more commonly in the GW field, a *false alarm*. If we incorrectly predict that a GW signal is absent when it is actually present, we refer to this as a *false negative* (FN), or a *false dismissal*. In machine learning (ML), these outcomes are typically summarized in a *confusion matrix*.

**Exercise 4:** Plot in a heatmap the confussion matrix.